# Data Preprocessing 2
Melissa Putri (1389438)

Validating data and converting data types

In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import count, col, hour, month, year, dayofweek, avg, lit, sum as spark_sum
import pandas as pd

# Create a spark session
spark = (
    SparkSession.builder.appName("Data Cleaning")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/23 14:02:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/23 14:02:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Load dataframe
uber_data = spark.read.parquet('../data/raw/cleaned_uber/')
lyft_data = spark.read.parquet('../data/raw/cleaned_lyft/')
uber_data.show(1)
lyft_data.show(1)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+------------------+---------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|is_accessible_flag|is_valid_record|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+----

In [3]:
uber_data.printSchema()
lyft_data.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: boolean (nullable = true)
 |-- shared_match_flag: boolean (nullable = true)
 |-- access_a_ride_flag: boolean (nullable = true)
 |-- wav_request_

### Removing Outliers

In [4]:
NUMERIC_COLS = ['trip_miles', 'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay']
def remove_outliers_iqr(dataframe, numerical_cols):
    for col in numerical_cols:
        quantiles = dataframe.approxQuantile(col, [0.25, 0.75], 0.01)
        Q1, Q3 = quantiles[0], quantiles[1]
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        dataframe = dataframe.filter((F.col(col) >= lower_bound) & (F.col(col) <= upper_bound))
    return dataframe

In [5]:
print(uber_data.count())
print(lyft_data.count())

172142764
65242190


In [6]:
uber_no_outliers = remove_outliers_iqr(uber_data, NUMERIC_COLS)
lyft_no_outliers = remove_outliers_iqr(lyft_data, NUMERIC_COLS)

24/08/23 14:02:17 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
print(uber_no_outliers.count())
print(lyft_no_outliers.count())

109499537


38383992


In [8]:
uber_data = uber_no_outliers
lyft_data = lyft_no_outliers

### Feature Engineering

In [9]:
# Create total_price column for Uber and Lyft data
uber_data = uber_data.withColumn('total_price', 
    col('base_passenger_fare') + col('tolls') + col('congestion_surcharge') + col('airport_fee'))
lyft_data = lyft_data.withColumn('total_price', 
    col('base_passenger_fare') + col('tolls') + col('congestion_surcharge') + col('airport_fee'))

In [10]:
# Create trip_duration_min column to convert trip time in minutes
uber_data = uber_data.withColumn('trip_duration_min', col('trip_time') / 60)
lyft_data = lyft_data.withColumn('trip_duration_min', col('trip_time') / 60)

In [11]:
# Extract date and time features
uber_data = uber_data.withColumn('hour', hour(col('request_datetime')))
uber_data = uber_data.withColumn('day_of_week', dayofweek(col('request_datetime')))
uber_data = uber_data.withColumn('month', month(col('request_datetime')))
uber_data = uber_data.withColumn('year', year(col('request_datetime')))

lyft_data = lyft_data.withColumn('hour', hour(col('request_datetime')))
lyft_data = lyft_data.withColumn('day_of_week', dayofweek(col('request_datetime')))
lyft_data = lyft_data.withColumn('month', month(col('request_datetime')))
lyft_data = lyft_data.withColumn('year', year(col('request_datetime')))

In [12]:
# Dollar per Mile for Uber
uber_data = uber_data.withColumn(
    "dollar_per_mile",
    F.when(
        (F.col("trip_miles") > 0) & (F.col("trip_miles").isNotNull()) & (F.col("base_passenger_fare") >= 0),  # Ensure no division by zero or nulls and fare is non-negative
        F.col("base_passenger_fare") / F.col("trip_miles")  # Calculate dollar per mile
    ).otherwise(0)  # Set to None if trip_miles is zero or null or fare is negative
)

# Dollar per Mile for Lyft
lyft_data = lyft_data.withColumn(
    "dollar_per_mile",
    F.when(
        (F.col("trip_miles") > 0) & (F.col("trip_miles").isNotNull()) & (F.col("base_passenger_fare") >= 0),  # Ensure no division by zero or nulls and fare is non-negative
        F.col("base_passenger_fare") / F.col("trip_miles")  # Calculate dollar per mile
    ).otherwise(0)  # Set to None if trip_miles is zero or null or fare is negative
)

# Show results for Uber
uber_data.select("trip_miles", "base_passenger_fare", "dollar_per_mile").show(10)

# Show results for Lyft
lyft_data.select("trip_miles", "base_passenger_fare", "dollar_per_mile").show(10)

+----------+-------------------+------------------+
|trip_miles|base_passenger_fare|   dollar_per_mile|
+----------+-------------------+------------------+
|      1.78|               8.87|4.9831460674157295|
|       6.3|              30.32|4.8126984126984125|
|      8.83|              35.98| 4.074745186862967|
|      7.69|               35.9| 4.668400520156046|
|      3.87|               16.3|4.2118863049095605|
|      1.55|                9.6| 6.193548387096774|
|      0.75|               8.03|10.706666666666665|
|      2.28|              13.33| 5.846491228070176|
|      0.91|               8.59|  9.43956043956044|
|      6.25|              22.78|            3.6448|
+----------+-------------------+------------------+
only showing top 10 rows

+----------+-------------------+------------------+
|trip_miles|base_passenger_fare|   dollar_per_mile|
+----------+-------------------+------------------+
|     8.527|               28.4|  3.33059692740706|
|     0.664|               9.16|13.795

## Aggregating External Dataset: Weather
Source: NCDC NOAA

In [13]:
weather_data = spark.read.csv("../data/landing/weather_data.csv", header=True, inferSchema=True)
print(f'Num of Instances: {weather_data.count()}')
print(f'Num of Columns/Features: {len(weather_data.columns)}')
weather_data = weather_data.withColumn("year", year("DATE"))
weather_data = weather_data.withColumn("month", month("DATE"))



Num of Instances: 986
Num of Columns/Features: 33


In [14]:
weather_data.show(5)


+-----------+---------+----------+---------+-------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+
|    STATION| LATITUDE| LONGITUDE|ELEVATION|               DATE|AWND|CDSD|CLDD|DP01|DP10|DSND|DSNW|DT00|DT32|DX32|DX70|DX90|EMNT|EMSD|EMSN|EMXP|EMXT|HDSD|HTDD|PRCP|SNOW|TAVG|TMAX|TMIN|WDF2|WDF5|WSF2|WSF5|year|month|
+-----------+---------+----------+---------+-------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+
|US1NJUN0028|40.708904|-74.321518|     37.5|2023-06-01 00:00:00|NULL|NULL|NULL|  13|   7|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|0.89|NULL|NULL|NULL| 3.3|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|2023|    6|
|US1NJUN0028|40.708904|-74.321518|     37.5|2023-07-01 00:00:00|NULL|NULL|NULL|  15|  10|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NU

24/08/23 14:03:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [15]:
# Aggregate Weather Dataset to Relevant/Features of Interest
weather_data_agg = weather_data.groupBy("year", "month").agg(
    avg("TAVG").alias("avg_temp"),
    avg("TMAX").alias("avg_max_temp"),
    avg("TMIN").alias("avg_min_temp"),
    spark_sum("PRCP").alias("total_precipitation"),
    avg("AWND").alias("average_wind")
)

In [16]:
weather_data_agg.show(12)
# Join with uber_data
uber_data = uber_data.join(weather_data_agg, on=["year", "month"], how="left")

# Join with lyft_data
lyft_data = lyft_data.join(weather_data_agg, on=["year", "month"], how="left")

+----+-----+------------------+------------------+------------------+-------------------+------------------+
|year|month|          avg_temp|      avg_max_temp|      avg_min_temp|total_precipitation|      average_wind|
+----+-----+------------------+------------------+------------------+-------------------+------------------+
|2023|    9| 68.88749999999999| 76.50625000000001|          61.26875|              827.6|7.4750000000000005|
|2023|    8|          73.96875|            81.725| 66.19375000000001|  404.6700000000001|7.1625000000000005|
|2024|    3| 45.94285714285714| 53.95714285714286| 37.90714285714286|  744.4499999999999|              9.85|
|2023|    7|             78.25| 86.66874999999997|            69.825| 487.70000000000016|            6.2375|
|2024|    5|             63.85|  72.2642857142857|55.442857142857136|  325.7800000000001|            7.0375|
|2023|    6| 68.69375000000001|          77.76875|59.618750000000006| 277.14000000000004|              7.05|
|2024|    2|       

In [17]:
# Filter data to ensure date range is from June 2023 to May 2024
uber_data = uber_data.filter(
    ((col("year") == 2023) & (col("month") >= 6)) |   # From June 2023 to December 2023
    ((col("year") == 2024) & (col("month") <= 5))     # From January 2024 to May 2024
)

lyft_data = lyft_data.filter(
    ((col("year") == 2023) & (col("month") >= 6)) |   # From June 2023 to December 2023
    ((col("year") == 2024) & (col("month") <= 5))     # From January 2024 to May 2024
)

# Show a few rows to verify the filtering
uber_data.show(5)
lyft_data.show(5)

+----+-----+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+------------------+---------------+-----------+------------------+----+-----------+------------------+-----------------+------------+------------------+-------------------+------------+
|year|month|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|is_accessible_flag|is_valid_record|total_price| trip_duration_min|hour|day_of_week|   dollar_per_m

## Standardizing Numeric Features

In [18]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.functions import vector_to_array

NUMERIC_COLS = ['trip_miles', 'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 
                'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay','total_price',
                'trip_duration_min', 'dollar_per_mile','avg_temp', 'avg_max_temp', 'avg_min_temp',
                'total_precipitation', 'average_wind']


def standardize_and_drop_original(dataframe, numerical_cols):
    """
    Function to standardize numeric columns in a PySpark DataFrame and drop the original columns.
    :param dataframe: PySpark DataFrame
    :param numerical_cols: List of numerical columns to standardize
    :return: DataFrame with standardized columns and original columns dropped
    """
    # Step 1: Create a feature vector from the numerical columns
    assembler = VectorAssembler(inputCols=numerical_cols, outputCol="features_vector")
    dataframe_vector = assembler.transform(dataframe)

    # Step 2: Apply StandardScaler to standardize the feature vector
    scaler = StandardScaler(inputCol="features_vector", outputCol="scaled_features", withMean=True, withStd=True)
    scaler_model = scaler.fit(dataframe_vector)
    dataframe_scaled = scaler_model.transform(dataframe_vector)

    # Step 3: Convert scaled feature vector into individual columns
    dataframe_scaled = dataframe_scaled.withColumn("scaled_array", vector_to_array(F.col("scaled_features")))
    
    for i, col_name in enumerate(numerical_cols):
        dataframe_scaled = dataframe_scaled.withColumn(col_name, F.col("scaled_array")[i])

    # Step 4: Drop intermediate columns and the original unscaled columns
    dataframe_final = dataframe_scaled.drop("features_vector", "scaled_features", "scaled_array")

    return dataframe_final

In [19]:
uber_std = standardize_and_drop_original(uber_data, NUMERIC_COLS)
lyft_std = standardize_and_drop_original(lyft_data, NUMERIC_COLS)

In [20]:
uber_data.show(5)
lyft_data.show(5)

+----+-----+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+------------------+---------------+-----------+------------------+----+-----------+------------------+-----------------+------------+------------------+-------------------+------------+
|year|month|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|is_accessible_flag|is_valid_record|total_price| trip_duration_min|hour|day_of_week|   dollar_per_m

## Export Data

In [21]:
# Export unscaled data for visualisation
uber_data.write.parquet('../data/curated/Uber_Cleaned')
lyft_data.write.parquet('../data/curated/Lyft_Cleaned')

In [22]:
# Export scaled data for model training
uber_std.write.parquet('../data/curated/Uber_Scaled')
lyft_std.write.parquet('../data/curated/Lyft_Scaled')

24/08/23 19:33:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 615623 ms exceeds timeout 120000 ms
24/08/23 19:33:04 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/23 19:33:05 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$